In [111]:
import numpy as np
import os

train_folder = os.getcwd() + '/Dataset/Image_features/Training'
validation_folder = os.getcwd() + '/Dataset/Image_features/PublicTest'
test_folder = os.getcwd() + '/Dataset/Image_features/PrivateTest'


def load_dataset():
    
    data_dict = dict()
    validation_dict = dict()
    test_dict = dict()
    
     # load Train set 
    data_dict['X'] = np.load(train_folder + '/landmarks.npy')
    data_dict['X'] = np.array([x.flatten() for x in data_dict['X']])
    data_dict['X'] = np.concatenate((data_dict['X'], np.load(train_folder + '/hog_features.npy')), axis=1)
    data_dict['Y'] = np.load(train_folder + '/labels.npy')
    
     # load validation set 
    validation_dict['X'] = np.load(validation_folder + '/landmarks.npy')
    validation_dict['X'] = np.array([x.flatten() for x in validation_dict['X']])
    validation_dict['X'] = np.concatenate((validation_dict['X'], np.load(validation_folder + '/hog_features.npy')), axis=1)
    validation_dict['Y'] = np.load(validation_folder + '/labels.npy')
    
    # load train set
    test_dict['X'] = np.load(test_folder + '/landmarks.npy')
    test_dict['X'] = np.array([x.flatten() for x in test_dict['X']])
    test_dict['X'] = np.concatenate((test_dict['X'], np.load(test_folder + '/hog_features.npy')), axis=1)
    test_dict['Y'] = np.load(test_folder + '/labels.npy')
    np.save(test_folder + "/lab.npy", test_dict['Y'])
    return data_dict, validation_dict, test_dict
    #return Train, Validation, Test Set 
    
    

In [112]:
import time
import argparse
import os
import sys
if sys.version_info >= (3, 0):
        import _pickle as cPickle
else:
        import cPickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#Hyperparameter

random_state = 0
epochs = 10000
epochs_during_hyperopt = 500
kernel = 'rbf'  # 'rbf', 'linear', 'poly' or 'sigmoid'
decision_function = 'ovr'  # 'ovo' for OneVsOne and 'ovr' for OneVsRest'
features = "landmarks_and_hog" # "landmarks" or "hog" or "landmarks_and_hog"
gamma = 'auto' # use a float number or 'auto' 


data, validation, test = load_dataset()


In [113]:
#Check the shape 
print(data['X'].shape)
print(data['Y'].shape)

(28709, 2728)
(28709,)


In [114]:
# Building Model
print( "building model...")
model = SVC(random_state=random_state, max_iter=epochs, kernel=kernel, decision_function_shape=decision_function, gamma=gamma)
print( "start training...")
print( "--")
print( "Training samples: {}".format(len(data['Y'])))
print( "Validation samples: {}".format(len(validation['Y'])))
print( "--")
start_time = time.time()
model.fit(data['X'], data['Y'])
training_time = time.time() - start_time
print( "training time = {0:.1f} sec".format(training_time))



building model...
start training...
--
Training samples: 28709
Validation samples: 3589
--
training time = 11472.8 sec


In [119]:
def evaluate(model, X, Y):
        predicted_Y = model.predict(X)
        accuracy = accuracy_score(Y, predicted_Y)
        return accuracy

In [120]:
print( "evaluating...")
#validation_accuracy = model.evaluate(validation['X'], validation['Y'],verbose=0)
validation_accuracy = evaluate(model, validation['X'],  validation['Y'])
print( "  - validation accuracy = {0:.1f}".format(validation_accuracy*100))

evaluating...
  - validation accuracy = 49.7


In [121]:
test_accuracy = evaluate(model, test['X'], test['Y'])
print( "  - test accuracy = {0:.1f}".format(test_accuracy*100))
print( "  - evalution time = {0:.1f} sec".format(time.time() - start_time))

  - test accuracy = 50.5
  - evalution time = 15517.8 sec


In [128]:
# once We will save this trained model then we can reuse it on other system.
from sklearn.externals import joblib

joblib.dump(model, 'SVMModel.joblib') 

['SVMModel.joblib']